In [1]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision import models
from torchvision.datasets import ImageNet
from torchvision import transforms
from torchvision.models.feature_extraction import create_feature_extractor, get_graph_node_names
from PIL import Image
import os
import sys
sys.path.append("/Users/tizianocausin/Desktop/backUp20240609/summer2025/ponce_lab/exp_set/python_scripts/src")
from sparsity_in_silico.sparsity_CNN import response_prob
#import imageio
import numpy as np
import h5py

In [2]:
os.getcwd()

'/Users/tizianocausin/Desktop/backUp20240609/summer2025/ponce_lab/exp_set/python_scripts/scripts'

In [3]:
#alexnet = models.alexnet(pretrained=True).eval()
alexnet = models.alexnet(weights=True).eval()
preprocess = transforms.Compose(
    [
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.3806, 0.4242, 0.3794], std=[0.2447, 0.2732, 0.2561]
        ),  # Normalization for pretrained model
    ]
)

#reader = imageio.get_reader(path2vid)
conv_layers = [
    "conv_layer1",
    "conv_layer4",
    "conv_layer7",
    "conv_layer9",
    "conv_layer11",
]
conv_layers_idx = [1, 4, 7, 9, 11]

fc_layers = ["fc_layer2", "fc_layer5"]
fc_layers_idx = [2, 5]

/Users/tizianocausin/Desktop/virtual_envs/ponce_env/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
feature_extractor = create_feature_extractor(alexnet, return_nodes={"features.0" : "1st", "features.1" : "2nd"})

In [5]:
#print(get_graph_node_names(alexnet))
layers = ['features.0', 'features.1', 'features.2', 'features.3', 'features.4', 'features.5', 'features.6', 'features.7', 'features.8', 'features.9', 'features.10', 'features.11', 'features.12', 'avgpool', 'classifier.0', 'classifier.1', 'classifier.2', 'classifier.3', 'classifier.4', 'classifier.5', 'classifier.6'] # took off the input and the "flatten" layer

In [10]:
for i in range(0, len(layers), 1):
    curr_layer = layers[i]
    print(curr_layer)

features.0
features.1
features.2
features.3
features.4
features.5
features.6
features.7
features.8
features.9
features.10
features.11
features.12
avgpool
classifier.0
classifier.1
classifier.2
classifier.3
classifier.4
classifier.5
classifier.6


In [74]:
print(alexnet)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [11]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
val_set = ImageNet(root="/Users/tizianocausin/Library/CloudStorage/OneDrive-SISSA/data_repo/exp_set_data/imagenet", split="val", transform=transform)

In [12]:

val_loader = DataLoader(
    val_set,
    batch_size=1,         
    shuffle=False,         
    num_workers=1
)

In [41]:
feats = {"1st" : [], "2nd":[]}

In [42]:
with torch.no_grad():  # Disable gradient tracking for evaluation
    for inputs, labels in val_loader:

        f = feature_extractor(inputs)
        #outputs = alexnet(inputs)           # Forward pass
        #_, preds = torch.max(outputs, 1)  # Get predicted class indices
        for layer, repr in f.items():
            print(layer)
            # flat_repr = repr.reshape(-1).detach().cpu().numpy().astype(np.float16)
            #feats[layer].append(flat_repr)
            feats[layer].append(repr)

        # Example: print first batch predictions
        
        break  # Remove this to go through the whole dataset

1st
2nd


In [55]:
my_arr = feats["1st"][0].cpu().numpy()

In [75]:
print(np.shape(my_arr))
print(np.max(my_arr, axis=(1,2)).shape)


(1, 64, 55, 55)
(1, 55)


In [19]:
for k, v in all_feats.items():
    print(k, v[0].size())

conv_layer1 torch.Size([193600])
conv_layer4 torch.Size([139968])
conv_layer7 torch.Size([64896])
conv_layer9 torch.Size([43264])
conv_layer11 torch.Size([43264])
fc_layer2 torch.Size([4096])
fc_layer5 torch.Size([4096])


In [11]:
a = response_prob(feats)
for k, v in a.items():
    print(k, v.size())

conv_layer1 torch.Size([3872])
conv_layer4 torch.Size([2799])
conv_layer7 torch.Size([1297])
conv_layer9 torch.Size([865])
conv_layer11 torch.Size([865])
fc_layer2 torch.Size([81])
fc_layer5 torch.Size([81])


In [70]:
path2res = "/Users/tizianocausin/Library/CloudStorage/OneDrive-SISSA/data_repo/exp_set_res/silico"
with h5py.File(f"{path2res}/dummy_feats_alexnet.h5", "a") as f:
# Iterate over dictionary items and save them in the HDF5 file
    for key, value in feats.items():
        f.create_dataset(
            key, data=value
        )  # Create a dataset for each key-value pair